In [57]:
    import pidgy.base, jinja2, IPython, sys

    import pidgy.base, jinja2, IPython, sys

In [89]:
    def active_types():
        shell = IPython.get_ipython()
        if shell:
            object = list(shell.display_formatter.active_types)
            object.insert(object.index('text/html'), object.pop(object.index('text/latex')))
            return reversed(object)
        return []

    def active_types():
        shell = IPython.get_ipython()
        if shell:
            object = list(shell.display_formatter.active_types)
            object.insert(object.index('text/html'), object.pop(object.index('text/latex')))
            return reversed(object)
        return []

In [98]:
    def environment():        
        environment = __import__('nbconvert').exporters.TemplateExporter().environment
        environment.loader.loaders.append(jinja2.FileSystemLoader('.'))
        environment.finalize = Finalize(parent=environment)
        environment.last_widgets = []
        return environment

    def environment():        
        environment = __import__('nbconvert').exporters.TemplateExporter().environment
        environment.loader.loaders.append(jinja2.FileSystemLoader('.'))
        environment.template_class = Template
        environment.finalize = Finalize(parent=environment)
        environment.last_widgets = []
        return environment

In [99]:
    class Finalize(pidgy.base.Trait):
        def normalize(self, key, object, metadata):            
            if key.startswith('image'):
                if 'svg' in key: return minify(object)
                width, height = metadata.get(key, {}).get('width'), metadata.get(key, {}).get('height')
                if isinstance(object, bytes):
                    object = __import__('base64').b64encode(object).decode('utf-8')
                object = F"""<img src="data:image/{key.partition('/')[2]};base64,{object}"/>"""
            if key == 'text/html': object = minify(object)
            return object
        
        def __call__(self, object):
            if isinteractive(object):
                if hasattr(object, 'value'): 
                    if object not in self.parent.last_widgets:
                        self.parent.last_widgets.append(object)
                    return self(object.value)

            datum = format(object)
            if isinstance(datum, tuple):
                data, metadata = datum
            else:
                data, metadata = datum, {}
            try: key = next(filter(data.__contains__, active_types()))
            except StopIteration: return str(object)
            return self.normalize(key, data[key], metadata)

    class Finalize(pidgy.base.Trait):
        def normalize(self, key, object, metadata):            
            if key.startswith('image'):
                if 'svg' in key: return minify(object)
                width, height = metadata.get(key, {}).get('width'), metadata.get(key, {}).get('height')
                if isinstance(object, bytes):
                    object = __import__('base64').b64encode(object).decode('utf-8')
                object = F"""<img src="data:image/{key.partition('/')[2]};base64,{object}"/>"""
            if key == 'text/html': object = minify(object)
            return object
        
        def __call__(self, object):
            if isinteractive(object):
                if hasattr(object, 'value'): 
                    self.parent.last_widgets.append(object)
                    return self(object.value)

            datum = format(object)
            if isinstance(datum, tuple):
                data, metadata = datum
            else:
                data, metadata = datum, {}
            try: key = next(filter(data.__contains__, active_types()))
            except StopIteration: return str(object)
            return self.normalize(key, data[key], metadata)

In [49]:
    def iswidget(object):
        if 'ipywidgets' in sys.modules:
            if isinstance(object, __import__('ipywidgets').Widget):
                return True
        return False

    def ispanel(object):
        if 'param' in sys.modules:
            if isinstance(object, __import__('param').Parameterized):
                return True
        return False

    def isinteractive(object):
        return iswidget(object) | ispanel(object)

    def iswidget(object):
        if 'ipywidgets' in sys.modules:
            if isinstance(object, __import__('ipywidgets').Widget):
                return True
        return False

    def ispanel(object):
        if 'param' in sys.modules:
            if isinstance(object, __import__('param').Parameterized):
                return True
        return False

    def isinteractive(object):
        return iswidget(object) | ispanel(object)